In [1]:
manualUpload = True #Define se irá usar upload do Drive (automático) ou da máquina (manual)

In [6]:
!pip install optuna --quiet

In [1]:
from keras.layers import Input, Dropout, BatchNormalization, CuDNNGRU, CuDNNLSTM, LSTM, Dense, Bidirectional, Activation
from keras.models import Sequential, Model

from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from IPython.display import display
from ipywidgets import IntProgress
from tensorflow import *
from scipy import stats

# import tensorflow_decision_forests 


import matplotlib.pyplot as plt
import tensorflow as tf

import seaborn as sns
import pandas as pd
import numpy as np
import optuna as op
import scipy as sp

import time
import io
import os

%matplotlib inline

In [2]:
# if manualUpload == False:
#     from google.colab import drive, files
#     drive.mount('/content/drive')

In [3]:
#Importação manual, caso necessária
# if not os.path.exists("TST.cod"):
#     if manualUpload == True:
#         upload = files.upload()
#         #decoding the files as uploaded will be a dictionary of keys (the file names) and values (the encoded file objects)
#         df_test = pd.read_table(io.StringIO(upload['TST.cod'].decode('utf-8')))

#     #Importação pelo Drive
#     else:
#         #decoding the files as uploaded will be a dictionary of keys (the file names) and values (the encoded file objects)
#         df_test = pd.read_table("/content/drive/MyDrive/Databases/TST.cod")
# else:
#     df_test = pd.read_table("TST.cod")

df_test = pd.read_table("TST.cod")

df_test.head()

,INDEX,UF_1,UF_2,UF_3,UF_4,UF_5,UF_6,UF_7,IDADE,SEXO_1,...,CEP4_7,CEP4_8,CEP4_9,CEP4_10,CEP4_11,CEP4_12,CEP4_13,CEP4_14,IND_BOM_1_1,IND_BOM_1_2
0,0,1,1,1,0,0,0,0,0.898745,1,...,0,1,1,1,0,0,0,0,1,0
1,1,1,1,0,1,0,0,0,0.847404,1,...,0,0,0,1,1,1,0,0,1,0
2,2,1,1,0,0,0,0,1,0.016513,0,...,0,1,0,0,1,0,0,1,0,1
3,3,1,1,0,1,0,0,0,0.688950,0,...,0,0,0,0,0,1,0,1,1,0
4,4,0,1,1,0,0,1,0,0.230114,1,...,0,1,0,0,1,0,1,0,1,0


In [15]:
df_test.head().T

,0,1,2,3,4
INDEX,0.0,1.0,2.0,3.0,4.0
UF_1,1.0,1.0,1.0,1.0,0.0
UF_2,1.0,1.0,1.0,1.0,1.0
UF_3,1.0,0.0,0.0,0.0,1.0
UF_4,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...
CEP4_12,0.0,1.0,0.0,1.0,0.0
CEP4_13,0.0,0.0,0.0,0.0,1.0
CEP4_14,0.0,0.0,1.0,1.0,0.0
IND_BOM_1_1,1.0,1.0,0.0,1.0,1.0


In [4]:
# #Importação manual, caso necessária
# if not os.path.exists("TRNcod.xls"):
#     if manualUpload == True:
#         upload = files.upload()
#         #decoding the files as uploaded will be a dictionary of keys (the file names) and values (the encoded file objects)
#         df_tr = pd.read_table(io.StringIO(upload['TRNcod.xls'].decode('utf-8')))

#     #Importação pelo Drive
#     else:
#         #decoding the files as uploaded will be a dictionary of keys (the file names) and values (the encoded file objects)
#         df_tr = pd.read_table("/content/drive/MyDrive/Databases/TRNcod.xls")
# else:
#     df_tr = pd.read_excel("TRNcod.xls")

df_tr = pd.read_table("TRNcod.xls") ##! <<<<<<<<<<<< READ_EXCEL NÃO FUNCIONA

# print(df_tr.columns.tolist())
df_tr.head()

,INDEX,UF_1,UF_2,UF_3,UF_4,UF_5,UF_6,UF_7,IDADE,SEXO_1,...,CEP4_7,CEP4_8,CEP4_9,CEP4_10,CEP4_11,CEP4_12,CEP4_13,CEP4_14,IND_BOM_1_1,IND_BOM_1_2
0,0,1,1,1,0,0,0,0,0.135098,1,...,0,0,1,1,0,1,1,1,0,1
1,1,1,0,1,0,0,1,0,0.273504,1,...,0,1,0,1,1,0,0,0,1,0
2,2,1,0,1,0,0,1,0,0.281910,0,...,1,1,0,0,0,0,1,0,1,0
3,3,1,1,1,0,0,0,0,0.225741,0,...,1,1,0,1,1,0,1,0,1,0
4,4,1,1,0,0,0,1,0,0.480403,0,...,1,1,1,0,0,1,0,1,1,0


In [17]:
'''
indarr = df_trein["IND_BOM_1_1"].to_numpy()
indarr2 = df_trein["IND_BOM_1_2"].to_numpy()

f = IntProgress(min = 0 , max = len(indarr)) # instantiate the bar
display(f) # display the bar

for i in range(len(indarr)):
    if indarr[i] == indarr2[i]:
        print(indarr[i], indarr2[i])
    if indarr[i] != 1 and indarr[i] != 0:
        print(f"indarr {i} = {indarr[i]}")
    if indarr2[i] != 1 and indarr2[i] != 0:
        print(f"indarr2 {i} = {indarr2[i]}")
    f.value += 1'''
#Utilizado para checar se IND_BOM_1_1 e IND_BOM_1_2 eram as duas classes separadas

'\nindarr = df_trein["IND_BOM_1_1"].to_numpy()\nindarr2 = df_trein["IND_BOM_1_2"].to_numpy()\n\nf = IntProgress(min = 0 , max = len(indarr)) # instantiate the bar\ndisplay(f) # display the bar\n\nfor i in range(len(indarr)):\n    if indarr[i] == indarr2[i]:\n        print(indarr[i], indarr2[i])\n    if indarr[i] != 1 and indarr[i] != 0:\n        print(f"indarr {i} = {indarr[i]}")\n    if indarr2[i] != 1 and indarr2[i] != 0:\n        print(f"indarr2 {i} = {indarr2[i]}")\n    f.value += 1'

In [5]:
if "IND_BOM_1_1" in df_tr:
    df_tr = df_tr.drop(columns = ["IND_BOM_1_1"])
if "INDEX" in df_tr:
    df_tr = df_tr.drop(columns = ["INDEX"])
if "IND_BOM_1_2" in df_tr:
    df_tr = df_tr.rename(columns = {"IND_BOM_1_2": "CLASSE_INDIV"})

df_tr.head()

#0 = Classe 1
#1 = Classe 2

,UF_1,UF_2,UF_3,UF_4,UF_5,UF_6,UF_7,IDADE,SEXO_1,NIVEL_RELACIONAMENTO_CREDITO01,...,CEP4_6,CEP4_7,CEP4_8,CEP4_9,CEP4_10,CEP4_11,CEP4_12,CEP4_13,CEP4_14,CLASSE_INDIV
0,1,1,1,0,0,0,0,0.135098,1,0.222222,...,0,0,0,1,1,0,1,1,1,1
1,1,0,1,0,0,1,0,0.273504,1,0.111111,...,0,0,1,0,1,1,0,0,0,0
2,1,0,1,0,0,1,0,0.281910,0,1.000000,...,0,1,1,0,0,0,0,1,0,0
3,1,1,1,0,0,0,0,0.225741,0,0.111111,...,0,1,1,0,1,1,0,1,0,0
4,1,1,0,0,0,1,0,0.480403,0,0.111111,...,1,1,1,1,0,0,1,0,1,0


In [6]:
df_tr_class_1 = df_tr[df_tr['CLASSE_INDIV'] == 0]
df_tr_class_2 = df_tr[df_tr['CLASSE_INDIV'] == 1]

df_tr_class_1_a = df_tr_class_1.sample(frac = 0.66)
df_tr_class_1_b = df_tr_class_1.drop(df_tr_class_1_a.index)

df_tr_class_2_a = df_tr_class_2.sample(frac = 0.66)
df_tr_class_2_b = df_tr_class_2.drop(df_tr_class_2_a.index)

df_tr_class_2_a = pd.concat([df_tr_class_2_a,
                     df_tr_class_2_a.sample(len(df_tr_class_1_a) - len(df_tr_class_2_a))],
                     ignore_index = True,
                     sort = False)

df_tr_class_2_b = pd.concat([df_tr_class_2_b,
                     df_tr_class_2_b.sample(len(df_tr_class_1_b) - len(df_tr_class_2_b))],
                     ignore_index = True,
                     sort = False)

df_trein = pd.concat([df_tr_class_1_a, df_tr_class_2_a])
df_valid = pd.concat([df_tr_class_1_b, df_tr_class_2_b])

print(len(df_tr_class_1_a), len(df_tr_class_2_b))

df_tr_class_1 = None
df_tr_class_2 = None

df_tr_class_1_a = None
df_tr_class_1_b = None
df_tr_class_2_a = None
df_tr_class_2_b = None

df_trein = shuffle(df_trein).reset_index().drop(columns = ["index"])
df_valid = shuffle(df_valid).reset_index().drop(columns = ["index"])

168365 86733


In [7]:
df_treinX = df_trein.drop(columns = ["CLASSE_INDIV"])
df_treinY = df_trein["CLASSE_INDIV"]
df_validX = df_valid.drop(columns = ["CLASSE_INDIV"])
df_validY = df_valid["CLASSE_INDIV"]

In [36]:
if "IND_BOM_1_1" in df_test:
    df_test = df_test.drop(columns = ["IND_BOM_1_1"])
if "INDEX" in df_test:
    df_test = df_test.drop(columns = ["INDEX"])
if "IND_BOM_1_2" in df_test:
    df_test = df_test.rename(columns = {"IND_BOM_1_2": "CLASSE_INDIV"})


df_test_x = df_test.drop(columns = ["CLASSE_INDIV"])
df_test_y = df_test["CLASSE_INDIV"]


In [8]:
patienceCallback = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 20)

In [ ]:
MLPmodel = Sequential([
    Dense(246, activation = 'relu'),
    Dense(10, activation = 'sigmoid'),
    ])

tf.keras.optimizers.Adam(
    learning_rate=0.01
)

loss_function = keras.losses.SparseCategoricalCrossentropy()

MLPmodel.compile(optimizer = 'adam',
      loss = loss_function,
      metrics=['accuracy'],
      )

file_path = 'training_2/cp.ckpt'
file_dir = os.path.dirname(file_path)

cb = tf.keras.callbacks.ModelCheckpoint(filepath=file_path,
                                        save_weights_only=True,
                                        verbose=0)

MLPmodel.fit(df_treinX, df_treinY, epochs = 10000, 
          batch_size = 2000, callbacks = [patienceCallback, cb],  
          validation_data = (df_validX, df_validY),
          verbose=2,

          )



### Acurácia muito baixa, precisamos passar pelo processo de tunagem

In [16]:
MLPmodel_2 = Sequential([
    Dense(128, activation = 'relu'),
    Dense(64, activation = 'sigmoid'),
    Dense(10, activation = 'sigmoid'),
    ])

loss_function = keras.losses.SparseCategoricalCrossentropy()


file_path = 'training_3/cp.ckpt'
file_dir = os.path.dirname(file_path)


# cb = tf.keras.callbacks.ModelCheckpoint(filepath=file_path,
#                                         save_weights_only=True,
#                                         verbose=0)

MLPmodel_2.compile(optimizer = 'nadam',
      loss = loss_function,
      metrics=['accuracy'])



MLPmodel_2.fit(df_treinX, df_treinY, epochs = 10, 
          batch_size = 2000, callbacks = [patienceCallback],  
          validation_data = (df_validX, df_validY),
          verbose=2,
          )

Epoch 1/10
169/169 - 3s - loss: 0.8052 - accuracy: 0.5827 - val_loss: 0.6617 - val_accuracy: 0.6200 - 3s/epoch - 16ms/step
Epoch 2/10
169/169 - 1s - loss: 0.6537 - accuracy: 0.6250 - val_loss: 0.6477 - val_accuracy: 0.6288 - 977ms/epoch - 6ms/step
Epoch 3/10
169/169 - 1s - loss: 0.6453 - accuracy: 0.6308 - val_loss: 0.6423 - val_accuracy: 0.6338 - 1s/epoch - 6ms/step
Epoch 4/10
169/169 - 1s - loss: 0.6406 - accuracy: 0.6350 - val_loss: 0.6403 - val_accuracy: 0.6352 - 993ms/epoch - 6ms/step
Epoch 5/10
169/169 - 1s - loss: 0.6374 - accuracy: 0.6378 - val_loss: 0.6369 - val_accuracy: 0.6373 - 975ms/epoch - 6ms/step
Epoch 6/10
169/169 - 1s - loss: 0.6352 - accuracy: 0.6397 - val_loss: 0.6358 - val_accuracy: 0.6381 - 985ms/epoch - 6ms/step
Epoch 7/10
169/169 - 1s - loss: 0.6334 - accuracy: 0.6411 - val_loss: 0.6343 - val_accuracy: 0.6401 - 1s/epoch - 6ms/step
Epoch 8/10
169/169 - 1s - loss: 0.6317 - accuracy: 0.6424 - val_loss: 0.6343 - val_accuracy: 0.6391 - 979ms/epoch - 6ms/step
Epoch 9/

In [18]:
MLPmodel_2.load_weights(file_path) ## carregar os pesos do treinamento anterior

In [37]:
loss, acc = MLPmodel_2.evaluate(df_test_x, df_test_y, verbose=2)

4055/4055 - 3s - loss: 1.4691 - accuracy: 0.5992 - 3s/epoch - 650us/step


In [40]:
predictions = MLPmodel_2.predict(df_test_x)

4055/4055 [==============================] - 3s 667us/step


In [48]:
test_results = predictions.argmax(axis=-1)

In [50]:
stats.ks_2samp(test_results, df_test_y)
# a = df_test.columns.to_list()


KstestResult(statistic=0.047142978270756086, pvalue=1.0192045277672721e-125)

In [ ]:
MLPmodel_2 = Sequential([
    Dense(64, activation = 'relu'),
    Dense(32, activation = 'sigmoid'),
    Dense(10, activation = 'sigmoid'),
    ])

MLPmodel_2.compile(optimizer = 'nadam',
      loss = 'mse',
      metrics=['accuracy'])

MLPmodel_2.fit(df_treinX, df_treinY, epochs = 10000, 
          batch_size = 1000, callbacks = [patienceCallback],  
          validation_data = (df_validX, df_validY))

In [36]:
RFmodel = tfdf.keras.RandomForestModel()

RFmodel.fit(tfdf.keras.pd_dataframe_to_tf_dataset(df_trein, label = "CLASSE_INDIV"), 
          callbacks = [patienceCallback])

NameError: name 'tfdf' is not defined

In [14]:
RFmodel.summary()

Model: "random_forest_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (243):
	ATIVIDADE_CONSUMIDOR_MERCADO_FINANCEIRO
	ATIVIDADE_EMAIL
	ATIVIDADE_ENDERECO
	ATIVIDADE_TELEFONE
	ATUALIZACAO_CONSUMIDOR_MERCADO_FINANCEIRO
	ATUALIZACAO_EMAIL
	ATUALIZACAO_ENDERECO
	BANCO_REST_IRPF_ULTIMA_1
	BANCO_REST_IRPF_ULTIMA_2
	BANCO_REST_IRPF_ULTIMA_3
	BANCO_REST_IRPF_ULTIMA_4
	BANCO_REST_IRPF_ULTIMA_5
	BANCO_REST_IRPF_ULTIMA_6
	BANCO_REST_IRPF_ULTIMA_7
	CAD_DEMOGRAFICO_VAR_0
	CAD_DEMOGRAFICO_VAR_1
	CAD_DEMOGRAFICO_VAR_10
	CAD_DEMOGRAFICO_VAR_11
	CAD_DEMOGRAFICO_VAR_13
	CAD_DEMOGRAFICO_VAR_14
	CAD_DEMOGRAFICO_VAR_15
	CAD_DEMOGRAFICO_VAR_16
	CAD_DEMOGRAFICO_VAR_17
	CAD_DEMOGRAFICO_VAR_19
	CAD_DEMOGRAFICO_VAR_2
	CAD_

In [17]:
RFmodel.compile(metrics = ["accuracy"])
RFmodel.evaluate(tfdf.keras.pd_dataframe_to_tf_dataset(df_valid, label = "CLASSE_INDIV"))

174/174 [==============================] - 55s 305ms/step - loss: 0.0000e+00 - accuracy: 0.6412


[0.0, 0.6412149667739868]

In [16]:
tfdf.model_plotter.plot_model_in_colab(RFmodel, tree_idx=0, max_depth = 3)